# AutoGen 基础示例

在此代码示例中，您将使用 [AutoGen](https://aka.ms/ai-agents/autogen) AI 框架创建一个基础代理。

此示例的目标是向您展示在后续代码示例中实现不同代理模式时将使用的步骤。


## 导入所需的 Python 包


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## 创建客户端

在本示例中，我们将使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 来访问 LLM。

`model` 被定义为 `gpt-4o-mini`。尝试将模型更改为 GitHub Models 市场中可用的其他模型，看看不同的结果。

作为一个简单测试，我们将运行一个简单的提示 - `法国的首都是哪里？`


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 定义代理

现在我们已经设置了 `client` 并确认其正常工作，接下来让我们创建一个 `AssistantAgent`。每个代理可以分配以下内容：
**name** - 一个简短的名称，用于在多代理流程中方便引用。
**model_client** - 你在前一步中创建的客户端。
**tools** - 代理可以用来完成任务的可用工具。
**system_message** - 定义任务、行为和 LLM 语气的元提示。

你可以修改 system message 来观察 LLM 的响应变化。我们将在第4课中详细讲解 `tools`。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## 运行代理

以下函数将运行代理。我们使用 `on_message` 方法通过新消息更新代理的状态。

在这个例子中，我们用来自用户的新消息更新状态，消息内容是 `"Plan me a great sunny vacation"`。

你可以更改消息内容，看看 LLM 会有怎样不同的响应。


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
